In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../../")
sys.path.append("../")

In [ ]:
import os
from os.path import join, basename
from glob import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
from typing import List, Any
from matplotlib import cm

from utils import load_syn_images, show_single_image, show_multiple_images, load_face_images, show_results, plot_surface, plot_grid_of_surfaces
from photometric.estimate_alb_nrm import estimate_alb_nrm
from photometric.check_integrability import check_integrability
from photometric.construct_surface import construct_surface
from photometric.photometric_stereo import photometric_stereo, photometric_stereo_face

### Yale face images

In [ ]:
image_dir="../yaleB02/"

In [ ]:
albedos = []
height_maps = []

In [ ]:
for path_type in ["column", "row", "average"]:
    albedo, normals, height_map, SE = photometric_stereo_face(
        image_dir=image_dir, path_type=path_type, return_cache=True, show=False
    )
    albedos.append(albedo)
    height_maps.append(height_map)

In [ ]:
%matplotlib inline

In [ ]:
show_multiple_images(
    albedos, grid=(1, 3), grayscale=True,
    xticks=False, yticks=False, save=False, path="../results/Yale_normals.png"
)

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(3, 6))

for i in range(3):
    ax[i].imshow(albedos[i], cmap="gray")
    ax[i].set_title("Column" * (i == 0) + "Row" * (i == 1) + "Average" * (i == 2))
    ax[i].set_xticks([])
    ax[i].set_yticks([])

plt.savefig("../results/yale_albedos.png", bbox_inches="tight")
plt.show()

In [ ]:
# for creating a responsive plot
%matplotlib widget

In [ ]:
plot_surface(height_maps[-1], set_lim=False, facecolors=np.dstack([albedos[-1].T, albedos[-1].T, albedos[-1].T]))

In [ ]:
facecolors = [np.dstack([x.T, x.T, x.T]) for x in albedos]

In [ ]:
plot_grid_of_surfaces(surfaces=height_maps, grid=(1, 3), facecolors=facecolors)

#### Analyzing face images

In [ ]:
[image_stack, scriptV, filenames] = load_face_images(image_dir)

In [ ]:
image_stack.shape

In [ ]:
# for creating a responsive plot
%matplotlib inline

In [ ]:
fig, ax = plt.subplots(8, 8, figsize=(15, 20))

for i in range(8):
    for j in range(8):
        idx = i * 8 + j
        ax[i, j].imshow(image_stack[:, :, idx], cmap="gray")
        ax[i, j].set_title(idx)
        ax[i, j].set_xticks([])
        ax[i, j].set_yticks([])

plt.show()

In [ ]:
annotations = {
    "Corrupted": [50, 54],
    "Light behind the face": [3, 7, 10, 13, 18, 25, 26, 33, 41, 43, 46, 48, 58],
    "Non-lambertian": [14, 19, 22, 38, 63],
}

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(3, 6))

for i, k in enumerate(list(annotations.keys())):
    ax[i, 0].imshow(image_stack[..., annotations[k][0]], cmap="gray")
    ax[i, 0].set_title(f"{k}", loc="left")
    ax[i, 0].set_xticks([])
    ax[i, 0].set_yticks([])

    ax[i, 1].imshow(image_stack[..., annotations[k][1]], cmap="gray")
    ax[i, 1].set_xticks([])
    ax[i, 1].set_yticks([])

plt.savefig("../results/yale_noisy_images.png", bbox_inches="tight")
plt.show()

In [ ]:
problematic = []
for x in annotations:
    problematic += annotations[x]

In [ ]:
new_image_stack = image_stack[..., list(set(range(64)) - set(problematic))]
new_scriptV = scriptV[list(set(range(64)) - set(problematic)),...]

In [ ]:
new_image_stack.shape, new_scriptV.shape

In [ ]:
albedo, normals = estimate_alb_nrm(new_image_stack, new_scriptV, shadow_trick=False)
p, q, SE = check_integrability(normals)
threshold = 0.005
SE[SE <= threshold] = float('nan')
height_map = construct_surface( p, q, path_type="average")
print('Number of outliers: %d\n' % np.sum(SE > threshold))
# show_results(albedo, normals, height_map, SE, set_lim=False)

In [ ]:
%matplotlib widget

In [ ]:
plot_surface(height_map, set_lim=False, facecolors=np.dstack([albedo.T, albedo.T, albedo.T]))